# Day 21: Fractal Art

You find a program trying to generate some art. It uses a strange process that involves repeatedly enhancing the detail of an image through a set of rules.

The image consists of a two-dimensional square grid of pixels that are either on (`#`) or off (`.`). The program always begins with this pattern:

In [1]:
import numpy as np


def parse(s):
    return np.array(
        [[1 if x == '#' else 0 for x in row] for row in s.split('/')])


def unparse(a):
    return '/'.join(''.join('#' if x else '.' for x in row) for row in a)


def pretty(a):
    return unparse(a).replace('/', '\n')


def parse_rulebook(s):
    rulebook = {}
    for line in s.strip().splitlines():
        lhs, rhs = map(parse, line.split(' => '))
        for cand in candidates(lhs):
            rulebook[unparse(cand)] = rhs
    return rulebook


def enhance(rulebook, img):
    N = len(img)
    bs = 2 if N % 2 == 0 else 3
    assert N % bs == 0
    n = N // bs
    img = img.reshape(n, bs, n, bs)
    new_img = np.empty((n, bs + 1, n, bs + 1))
    for i in range(n):
        new_row = []
        for j in range(n):
            src = unparse(img[i, :, j, :])
            dest = rulebook[src]
            new_img[i, :, j, :] = dest
    return new_img.reshape((bs + 1) * n, (bs + 1) * n)


def enhance_repeatedly(rulebook, n, img=None):
    if img is None:
        img = INITIAL
    for _ in range(n):
        img = enhance(rulebook, img)
    return img


INITIAL = parse('.#./..#/###')
print(pretty(INITIAL))

.#.
..#
###


Because the pattern is both 3 pixels wide and 3 pixels tall, it is said to have a size of 3.

Then, the program repeats the following process:

- If the size is evenly divisible by 2, break the pixels up into 2x2 squares, and convert each 2x2 square into a 3x3 square by following the corresponding enhancement rule.
- Otherwise, the size is evenly divisible by 3; break the pixels up into 3x3 squares, and convert each 3x3 square into a 4x4 square by following the corresponding enhancement rule.
Because each square of pixels is replaced by a larger one, the image gains pixels and so its size increases.

The artist's book of enhancement rules is nearby (your puzzle input); however, it seems to be missing rules. The artist explains that sometimes, one must rotate or flip the input pattern to find a match. (Never rotate or flip the output pattern, though.) Each pattern is written concisely: rows are listed as single units, ordered top-down, and separated by slashes. For example:

In [2]:
assert pretty(parse('../.#')) == """
..
.#
""".strip()

assert pretty(parse('.#./..#/###')) == """
.#.
..#
###
""".strip()

assert pretty(parse('#..#/..../#..#/.##.')) == """
#..#
....
#..#
.##.
""".strip()

When searching for a rule to use, rotate and flip the pattern as necessary. For example, all of the following patterns match the same rule:

```
.#.   .#.   #..   ###
..#   #..   #.#   ..#
###   ###   ##.   .#.
```

In [3]:
def hflip(img):
    return np.fliplr(img)


def rotate(img):
    return hflip(np.transpose(img))


def candidates(rule):
    for _ in range(4):
        yield rule
        yield hflip(rule)
        rule = rotate(rule)


def match(rule, img):
    for cand in candidates(rule):
        if (cand == img).all():
            return True
    return False


imgs = ['.#./..#/###', '.#./#../###', '#../#.#/##.', '###/..#/.#.']
for i in imgs:
    for j in imgs:
        assert match(parse(i), parse(j))

Suppose the book contained the following two rules:

In [5]:
example = parse_rulebook("""
../.# => ##./#../...
.#./..#/### => #..#/..../..../#..#
""");

As before, the program begins with this pattern:

In [6]:
print(pretty(INITIAL))

.#.
..#
###


The size of the grid (3) is not divisible by 2, but it is divisible by 3. It divides evenly into a single square; the square matches the second rule, which produces:

In [7]:
assert pretty(enhance_repeatedly(example, 1)) == """
#..#
....
....
#..#
""".strip()

The size of this enhanced grid (4) is evenly divisible by 2, so that rule is used. It divides evenly into four squares:

```
#.|.#
..|..
--+--
..|..
#.|.#
```

Each of these squares matches the same rule (`../.# => ##./#../...`), three of which require some flipping and rotation to line up with the rule. The output for the rule is the same in all four cases:

```
##.|##.
#..|#..
...|...
---+---
##.|##.
#..|#..
...|...
```

Finally, the squares are joined into a new grid:


In [8]:
assert pretty(enhance_repeatedly(example, 2)) == """
##.##.
#..#..
......
##.##.
#..#..
......
""".strip()

Thus, after 2 iterations, the grid contains 12 pixels that are on.

In [9]:
assert unparse(enhance_repeatedly(example, 2)).count('#') == 12

How many pixels stay on after 5 iterations?

In [10]:
puzzle = parse_rulebook(open('21.input').read())
unparse(enhance_repeatedly(puzzle, 5)).count('#')

203

## Part Two

How many pixels stay on after 18 iterations?

In [11]:
%%time
unparse(enhance_repeatedly(puzzle, 18)).count('#')

CPU times: user 6.62 s, sys: 105 ms, total: 6.73 s
Wall time: 6.79 s


3342470